# Project 4 

Outline:

* scraping data

* binning: by salary

* parsing data: tokenizing, bigrams, most_common words for each bin 

* building model

* evaluating model


Downloading the whole data for US. 09/29/2016

In [ ]:
df

In [128]:
# check one entry to verify contents
print df.iloc[6]

company                                                             Cisco Systems, Inc.
job_link        http://www.indeed.com/rc/clk?jk=6cb402d057d00c35&fccid=dfc44f3b8c44a6db
job_location                                                               San Jose, CA
job_posted                                                                  Just posted
job_title                                       Data Scientist - Technology and Quality
Name: 6, dtype: object


In [129]:
# Save the result to CSV
df.to_csv('./indeed_companies.csv', encoding='utf-8')

Downloading the whole data for Australia. 09/29/2016

In [130]:
# load the library


df_au = pd.DataFrame()   # create a new data frame
pd.set_option('max_colwidth',500)    # to remove column limit (default is 50. Otherwise, we'll lose some info)


# indeed.com's query url
base_url = 'http://au.indeed.com/jobs?q=Data+Scientist&jt=fulltime'
sort_by = 'date'          # sort by data
start_from = '&start='    # start page number

for page in range(1,10): # page from 1 to 100 
    page = (page-1) * 10   # pagination
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
    contents = BeautifulSoup(urllib2.urlopen(url), "lxml") 

    contentsElements_au = contents.find_all('div', attrs={"class" : '  row  result'})
    cont_au = contents.find('div', attrs = {'class': 'lastRow  row  result'}) ##just added last job on each page
    contentsElements_au.append(cont_au)
    # each row contains 1 job

    # extract job information
    for elem in contentsElements_au: 
        try:
            company_au = elem.find('span', attrs={'itemprop':'name'}).getText().strip() #strips off whitespace 
        except AttributeError:
            company = '-'
        job_title_au = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        home_url_au = "http://www.indeed.com"
        job_link_au = "%s%s" % (home_url,elem.find('a').get('href'))
        job_addr_au = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()
        job_posted_au = elem.find('span', attrs={'class': 'date'}).getText()

           # add a record of job info to our data frame

        df_au = df_au.append({'company': company_au, 
                        'job_title': job_title_au, 
                        'job_link': job_link_au, 
                        'job_posted': job_posted_au,
                        'job_location': job_addr_au
                       }, ignore_index=True)
df_au

Save the result to CSV
df_au.to_csv('./indeed_companies_au.csv', encoding='utf-8')

jobs_data = pd.read_csv('indeed_companies.csv')
jobs_data_au = pd.read_csv('indeed_companies_au.csv')
frames = [jobs_data, jobs_data_au]
result = pd.concat(frames, axis = 0, ignore_index = True)
result

.

In [51]:
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''      

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()

    return None


def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib2, requests


url_template = "http://www.indeed.com/jobs?q=data+scientist+${}%2C000&l=USA&start={}"
max_results_per_city = 5000 # Set this to a high-value (5000) to generate more results. 

# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

#city = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()

rows = []
for salary in set([ '55-70', '70-85', '85-100', '100-115']):
    for start in range(1,1000): 
        r = requests.get(url_template.format(salary, start))
        soup = BeautifulSoup(r.content) 
        results = soup.findAll('div', { "class" : "result" })
        for result in results:
            if result:
                row = {}
                row['title'] = get_title_from_result(result)
                row['company'] = get_company_from_result(result)
                row['summary'] = get_summary_from_result(result)
                row['city'] = get_location_from_result(result) 
                row['bin'] = salary
            #row['salary'] = get_salary_from_result(result)
                rows.append(row)
                

C:\Users\voyo\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
result.to_csv('./indeed_companies.csv', encoding='utf-8')

In [50]:
result = pd.DataFrame.from_records(rows)
#result.set_option('max_colwidth',500) # create a new data frame
result.drop_duplicates(subset = "summary")
result

AttributeError: 'DataFrame' object has no attribute 'set_option'

.

In [49]:
df = pd.read_csv('indeed_companies.csv')
df[df.bin =="55-70"]

,Unnamed: 0,bin,city,company,summary,title
29970,29970,55-70,"Rockville, MD",ICF,Experience analyzing data from large social su...,Data Analyst
29971,29971,55-70,"Wellesley, MA 02481",Aetna,"The Data Scientist will develop, implement and...",Data Scientist Statistician
29972,29972,55-70,"Newton, MA","Thermalin Diabetes, LLC",Knack for communicating stories through data v...,"Data Scientist - Newton, MA (will work in Bost..."
29973,29973,55-70,"Framingham, MA",Staples,: Qualifications Job : Location(s) : Other Loc...,"Data Scientist, Framingham, MA"
29974,29974,55-70,"Hartford, CT 06156",Aetna,Ability to work with large data sets from mult...,Data Scientist Analyst
29975,29975,55-70,"Cambridge, MA",KAYAK.com,"Investigate data problems, identify patterns. ...",Data Scientist
29976,29976,55-70,"Springfield, VA",Concurrent Technologies Corporation,Associate Data Scientist. CTC has an opportuni...,Associate Data Scientist
29977,29977,55-70,"Raleigh, NC","Resolvit, LLC","Technical knowledge regarding data models, dat...",Data Scientist
29978,29978,55-70,"Cambridge, MA",Massachusetts General Hospital(MGH),"In addition, this position involves data analy...","Research Scientist, Sr. (Phd)"
29979,29979,55-70,"Alpharetta, GA",krg technology inc,Data scientist with either a Masters or PhD. S...,Data Scientist


In [6]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39960 entries, 0 to 39959
Data columns (total 5 columns):
bin        39960 non-null object
city       39960 non-null object
company    39960 non-null object
summary    39960 non-null object
title      39960 non-null object
dtypes: object(5)
memory usage: 1.5+ MB


In [7]:
%matplotlib inline
import seaborn
result.bin.value_counts()

85-100     9990
55-70      9990
100-115    9990
70-85      9990
Name: bin, dtype: int64

In [223]:
#result.company.unique()

In [8]:
import nltk
import nltk.stem
import nltk.stem.porter
import nltk.stem.snowball
snowball = nltk.stem.snowball.SnowballStemmer("english")
from nltk.corpus import stopwords
import re

Extracting data from DataFrame and Cleaning it from numbers, charactters, stopwords.

In [35]:
junior = str(result[result['bin'].isin(['55-70'])].summary) #subsetting over categories(bins)
mid_low = str(result[result['bin'].isin(['70-85'])].summary)
mid_high = str(result[result['bin'].isin(['85-100'])].summary)
senior = str(result[result['bin'].isin(['100-115'])].summary)

junior_clean = re.sub("[^a-zA-Z]", " ", junior)
mid_low_clean = re.sub("[^a-zA-Z]", " ", mid_low)
mid_high_clean = re.sub("[^a-zA-Z]", " ", mid_high)
senior_clean = re.sub("[^a-zA-Z]", " ", senior)

#junior_clean

stop = set(stopwords.words('english'))
stop.update([',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '...'])

junior_words = [w for w in (nltk.word_tokenize(junior))]# if w.lower() not in stop]
mid_low_words = [w for w in (nltk.word_tokenize(mid_low_clean)) if w.lower() not in stop]
mid_high_words = [w for w in (nltk.word_tokenize(mid_high_clean)) if w.lower() not in stop]
senior_words = [w for w in (nltk.word_tokenize(senior_clean)) if w.lower() not in stop]


len(junior_words)

611

In [43]:
junior

'29970    Experience analyzing data from large social su...\n29971    The Data Scientist will develop, implement and...\n29972    Knack for communicating stories through data v...\n29973    : Qualifications Job : Location(s) : Other Loc...\n29974    Ability to work with large data sets from mult...\n29975    Investigate data problems, identify patterns. ...\n29976    Associate Data Scientist. CTC has an opportuni...\n29977    Technical knowledge regarding data models, dat...\n29978    In addition, this position involves data analy...\n29979    Data scientist with either a Masters or PhD. S...\n29980    Experience analyzing data from large social su...\n29981    The Data Scientist will develop, implement and...\n29982    Knack for communicating stories through data v...\n29983    : Qualifications Job : Location(s) : Other Loc...\n29984    Ability to work with large data sets from mult...\n29985    Investigate data problems, identify patterns. ...\n29986    Associate Data Scientist. CTC 

In [37]:
from nltk.probability import FreqDist
fdist_junior = FreqDist(junior_words)
jun_features = sorted(w for w in set(junior_words) if len(w) > 2 and fdist_junior[w] > 3)
jun_features

['...',
 'Associate',
 'CLINICAL',
 'Data',
 'LAB',
 'Scientist',
 'The',
 'and',
 'data',
 'for',
 'from',
 'large',
 'summary',
 'the',
 'will',
 'with']

In [13]:
from nltk.probability import FreqDist
fdist_mid_low = FreqDist(mid_low_words)
sorted(w for w in set(mid_low_words) if len(w) > 2 and fdist_mid_low[w] > 3)

['Data',
 'Experience',
 'Scientist',
 'SiteLock',
 'data',
 'large',
 'sets',
 'working']

In [228]:
from nltk.probability import FreqDist
fdist_mid_high = FreqDist(mid_high_words)
sorted(w for w in set(mid_high_words) if len(w) > 2 and fdist_mid_high[w] > 5)

['Data',
 'Experience',
 'Operations',
 'Scientist',
 'Seeking',
 'Senior',
 'analysis',
 'analytics',
 'applying',
 'build',
 'business',
 'cleansing',
 'data',
 'database',
 'engineers',
 'experience',
 'join',
 'large',
 'learning',
 'machine',
 'mining',
 'modeling',
 'models',
 'new',
 'quality',
 'seeking',
 'sources',
 'statistical',
 'support',
 'team',
 'test',
 'tools']

In [23]:
from nltk.probability import FreqDist
fdist_senior = FreqDist(senior_words)
sorted(w for w in set(senior_words) if len(w) > 2 and fdist_senior[w] > 5)

['Data', 'Scientist', 'data']

Now lets have a look at bigrams and trigrams as they are more meaningful. In the next few steps I am going to highlight most common bigrams and trigrams for each of 4 categories:

In [240]:
jun_pairs = [ " ".join(pair) for pair in nltk.bigrams(junior_words)]
jun_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(junior_words)])
fdist_junpairs = FreqDist(jun_pairs)
fdist_jun_trigrams = FreqDist(jun_trigrams)


sorted(w for w in set(jun_pairs) if fdist_junpairs[w] > 5)
#sorted(w for w in set(jun_trigrams) if fdist_jun_trigrams[w] > 5)

['Investigate data',
 'analyzing investigating',
 'data analysis',
 'data mining',
 'data models',
 'data parsing',
 'data problems',
 'identify patterns',
 'interpreting modeling',
 'investigating interpreting',
 'love working',
 'mining segmentation',
 'modeling extracting',
 'parsing analyzing',
 'patterns person',
 'person love',
 'problems identify',
 'working data']

In [239]:
mid_low_pairs = [ " ".join(pair) for pair in nltk.bigrams(mid_low_words)]
mid_low_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(mid_low_words)])
fdist_mid_low = FreqDist(mid_low_pairs)
fdist_mid_low_trigrams = FreqDist(mid_low_trigrams)


sorted(w for w in set(mid_low_pairs) if fdist_mid_low[w] > 5)
#sorted(w for w in set(mid_low_trigrams) if fdist_mid_low_trigrams[w] > 2)

['Data Scientist',
 'Physics Division',
 'clinical data',
 'data analysis',
 'data quality',
 'data sets',
 'data sources',
 'data using',
 'ensure data',
 'large data',
 'successful candidate']

In [245]:
mid_high_pairs = [ " ".join(pair) for pair in nltk.bigrams(mid_high_words)]
mid_high_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(mid_high_words)])
fdist_mid_high = FreqDist(mid_high_pairs)
fdist_mid_high_trigrams = FreqDist(mid_high_trigrams)


sorted(w for w in set(mid_high_pairs) if fdist_mid_high[w] > 5)
#sorted(w for w in set(mid_high_trigrams) if fdist_mid_high_trigrams[w] > 2)

['Data Scientist',
 'Experience data',
 'data analysis',
 'data mining',
 'machine learning',
 'mining data',
 'new data',
 'quality data']

In [250]:
senior_pairs = [ " ".join(pair) for pair in nltk.bigrams(senior_words)]
senior_trigrams = sorted([ " ".join(pair) for pair in nltk.trigrams(senior_words)])
fdist_senior = FreqDist(senior_pairs)
fdist_senior_trigrams = FreqDist(senior_trigrams)


sorted(w for w in set(senior_pairs) if fdist_senior[w] > 3)
#sorted(w for w in set(senior_trigrams) if fdist_senior_trigrams[w] > 2)

['Data Analysis',
 'Data Science',
 'Data Scientist',
 'Data Scientists',
 'Field Data',
 'Scientist experience',
 'data management',
 'data mining',
 'data scientist',
 'data scientists',
 'end end',
 'experience leading',
 'highly motivated',
 'looking highly']

Now it seems that towards senior level words as "expert", "management", "lead" appear more frequently. While words like "parsing" and "extracting" are more common for junior category

In [232]:
lr = sklearn.linear_model.LogisticRegression()
lr.fit(titanic[['Age']] ,titanic.Survived)
probabilities = lr.predict_proba([ [age] for age in range(0,101)  ])
matplotlib.pyplot.plot(probabilities)
probabilities

NameError: name 'sklearn' is not defined